In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
import math
import logging
import yaml
import warnings
import gc
sys.path.append('../')

from shapely import MultiPoint, Point, MultiLineString, LineString, MultiPolygon, Polygon, GeometryCollection

In [2]:
# Set the option to display all rows
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
# global threshold to display all uid in error message
DISPLAY_THRESH = 10

In [3]:
# global threshold for minimum % of COMPLETE activity status 
COMPLETE_THRESH = 0.6

## Load file path from config yaml file

In [56]:
# load config file path yaml
# please change the yaml file paths first 
with open("..\config.yaml", 'r') as stream:
    config_inputs = yaml.safe_load(stream)


<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\sky\AppData\Local\Temp\ipykernel_200020\848487185.py:3: SyntaxWarning: invalid escape sequence '\c'
  with open("..\config.yaml", 'r') as stream:


In [27]:
error_count = 0

# concat error print messages
error_msg = ''

## CNRA

In [6]:
# load file path from config file
# you can also manually change the string here
# example:
# input_gdb_path = 'agency_folder/agency.gdb'
cnra_input_gdb_path = config_inputs['sources']['cnra']['input']['gdb_path']
cnra_polygon_layer_name = config_inputs['sources']['cnra']['input']['polygon_layer_name']
cnra_line_layer_name = config_inputs['sources']['cnra']['input']['line_layer_name']
cnra_point_layer_name = config_inputs['sources']['cnra']['input']['point_layer_name']
cnra_project_polygon_layer_name = config_inputs['sources']['cnra']['input']['project_layer_name']
cnra_activity_layer_name =config_inputs['sources']['cnra']['input']['activity_layer_name']

In [7]:
# check file type
valid_file_types = ['gdb']
try: 
    assert cnra_input_gdb_path[-3:] in valid_file_types
except: 
    error_count += 1
    warnings.warn("CNRA input is not the correct file type") 
    error_msg += "CNRA input is not the correct file type" + "\n"

    print("    Required file type: ", valid_file_types)
    print("    Input file type: ", cnra_input_gdb_path[-3:])
    error_msg += "    Required file type: " + str(valid_file_types) + "\n"
    error_msg += "    Input file type: " + str(cnra_input_gdb_path[-3:]) + "\n"

In [8]:
print(error_msg)

In [9]:
# check input layers
layers = [cnra_polygon_layer_name,
         cnra_line_layer_name,
         cnra_point_layer_name,
         cnra_project_polygon_layer_name,
         cnra_activity_layer_name]

input_layers_dict = gpd.list_layers(cnra_input_gdb_path).set_index('name')
# check if all input layer names are correct, exist in input gdb file
try:
    assert set(layers) == set(input_layers_dict.index)
except:
    error_count += 1
    warnings.warn("Not all input layers exist")
    error_msg += "Not all input layers exist" + "\n"
    
    print("    Provided layer names: ", layers)
    print("    Existing layer names: ", list(input_layers_dict.index))
    error_msg += "    Provided layer names: " + str(layers) + "\n"
    error_msg += "    Existing layer names: " + str(input_layers_dict.index) + "\n"

In [10]:
# required columns
CNRA_COLUMNS = [
    'county', 'created_date', 'created_user', 'geometry', 'globalid',
    'globalid_text', 'in_wui', 'last_edited_date', 'last_edited_user', 'org_admin_t',
    'primary_objective', 'primary_ownership_group', 'projectid', 'projectid_user',
    'projectname_', 'region', 'retreatment_date_est', 'secondary_objective',
    'tertiary_objective', 'treatment_area', 'treatment_end', 'treatment_name',
    'treatment_start', 'treatment_status', 'trmtid_user', 'validationstatus']

CNRA_PROJECT_COLUMNS = [
    'administering_org', 'agency',
    'created_date', 'created_user', 'geometry', 'globalid', 'globalid_text',
    'implementing_org', 'last_edited_date', 'last_edited_user', 'latitude',
    'longitude', 'org_admin_p', 'primary_funding_org', 'primary_funding_source',
    'project_contact', 'project_email', 'project_end', 'project_name',
    'project_start', 'project_status', 'projectid_user', 'validationstatus']

CNRA_ACTIVITY_COLUMNS = [
    'activid_user', 'activity_cat', 'activity_description',
    'activity_end', 'activity_name', 'activity_prct', 'activity_quantity',
    'activity_start', 'activity_status', 'activity_uom', 'admin_org_name',
    'broad_vegetation_type', 'bvt_userd', 'counts_to_mas', 'created_date',
    'created_user', 'globalid', 'implem_org_name', 'last_edited_date',
    'last_edited_user', 'org_admin_a', 'primary_fund_org_name',
    'primary_fund_src_name', 'primary_objective', 'residue_fate',
    'residue_fate_quantity', 'residue_fate_units', 'secondary_fund_org_name',
    'secondary_fund_src_name', 'tertiary_fund_org_name', 'tertiary_fund_src_name',
    'treatmentid_ln', 'treatmentid_poly', 'treatmentid_pt', 'trmt_geom',
    'trmtid_user']

### CNRA Point

In [11]:
# check if input layer geometry types are correct
try:
    assert input_layers_dict.loc[cnra_point_layer_name, 'geometry_type'] in ['Point', 'MultiPoint']
except:
    error_count += 1
    warnings.warn("Input geometry type incorrect")
    error_msg += "Input geometry type incorrect" + "\n"
    
    print("    Valid geom types: ", ['Point', 'MultiPoint'])
    print("    Input geom types: ", input_layers_dict.loc[cnra_point_layer_name, 'geometry_type'])
    error_msg += "    Valid geom types: " + str(['Point', 'MultiPoint']) + "\n"
    error_msg += "    Input geom types: " + input_layers_dict.loc[cnra_point_layer_name, 'geometry_type'] + "\n"    

In [12]:
# load data to gdf
input_gdf = gpd.read_file(cnra_input_gdb_path,
                         driver='OpenFileGDB',
                         layer=cnra_point_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [13]:
# schema check
try:
    assert len(set(CNRA_COLUMNS) - set(input_gdf.columns)) == 0
except:
    error_count += 1
    warnings.warn("Not all required fields are present in input")
    error_msg += "Not all required fields are present in input" + "\n"
    
    print("    Missing required fields: ", set(CNRA_COLUMNS) - set(input_gdf.columns))
    error_msg += "    Missing required fields: "+ str(set(CNRA_COLUMNS) - set(input_gdf.columns)) + "\n"

In [14]:
# uid check
try:
    assert len(input_gdf.globalid.unique()) == len(input_gdf.globalid)
except:
    error_count += 1

    warnings.warn("Exist duplicate unique ID")
    error_msg += "Exist duplicate unique ID" + "\n"
    
    display(input_gdf[input_gdf.duplicated(subset=['globalid'])])
    error_msg += str(list(input_gdf.loc[input_gdf.duplicated(subset=['globalid']), 'globalid'])) + "\n"

In [15]:
# invalid geometry check
valid_col = input_gdf.is_valid.apply(lambda x: "VALID" if x else "REPAIRED")
try:
    assert sum(input_gdf.is_valid) == len(input_gdf)
except:
    error_count += 1
    warnings.warn('Not all input geometries are valid')
    error_msg += "Not all input geometries are valid" + "\n"
    print("Attempt to repair geometry: ")
    repaired_geom = input_gdf.make_valid()
    # not fixable geometries
    # in pipeline these will be reduced to Point
    print(f"    Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries")
    error_msg += f"    Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries" + "\n"
    repaired_geom.loc[repaired_geom.geom_type == 'GeometryCollection'] == input_gdf.geometry.loc[repaired_geom.geom_type == 'GeometryCollection']
    valid_col.loc[repaired_geom.geom_type == 'GeometryCollection'] == "INVALID"
    
input_gdf['ValidGeometry'] = valid_col

In [ ]:
del input_gdf
gc.collect()

### CNRA Line

In [41]:
# check if input layer geometry types are correct
try:
    assert input_layers_dict.loc[cnra_line_layer_name, 'geometry_type'] in ['LineString', 'MultiLineString']
except:
    error_count += 1
    warnings.warn("Input geometry type incorrect")
    error_msg += "Input geometry type incorrect" + "\n"
    
    print("    Valid geom types: ", ['LineString', 'MultiLineString'])
    print("    Input geom types: ", input_layers_dict.loc[cnra_line_layer_name, 'geometry_type'])
    error_msg += "    Valid geom types: " + str(['LineString', 'MultiLineString']) + "\n"
    error_msg += "    Input geom types: " + input_layers_dict.loc[cnra_line_layer_name, 'geometry_type'] + "\n"    

In [ ]:
# load data to gdf
input_gdf = gpd.read_file(cnra_input_gdb_path,
                         driver='OpenFileGDB',
                         layer=cnra_line_layer_name)

In [ ]:
# schema check
try:
    assert len(set(CNRA_COLUMNS) - set(input_gdf.columns)) == 0
except:
    error_count += 1
    warnings.warn("Not all required fields are present in input")
    error_msg += "Not all required fields are present in input" + "\n"
    
    print("    Missing required fields: ", set(CNRA_COLUMNS) - set(input_gdf.columns))
    error_msg += "    Missing required fields: "+ str(set(CNRA_COLUMNS) - set(input_gdf.columns)) + "\n"

In [ ]:
# uid check
try:
    assert len(input_gdf.globalid.unique()) == len(input_gdf.globalid)
except:
    error_count += 1

    warnings.warn("Exist duplicate unique ID")
    error_msg += "Exist duplicate unique ID" + "\n"
    
    display(input_gdf[input_gdf.duplicated(subset=['globalid'])])
    error_msg += str(list(input_gdf.loc[input_gdf.duplicated(subset=['globalid']), 'globalid'])) + "\n"

In [ ]:
# invalid geometry check
valid_col = input_gdf.is_valid.apply(lambda x: "VALID" if x else "REPAIRED")
try:
    assert sum(input_gdf.is_valid) == len(input_gdf)
except:
    error_count += 1
    warnings.warn('Not all input geometries are valid')
    error_msg += "Not all input geometries are valid" + "\n"
    print("Attempt to repair geometry: ")
    repaired_geom = input_gdf.make_valid()
    # not fixable geometries
    # in pipeline these will be reduced to Point
    print(f"    Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries")
    error_msg += f"    Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries" + "\n"
    repaired_geom.loc[repaired_geom.geom_type == 'GeometryCollection'] == input_gdf.geometry.loc[repaired_geom.geom_type == 'GeometryCollection']
    valid_col.loc[repaired_geom.geom_type == 'GeometryCollection'] == "INVALID"
    
input_gdf['ValidGeometry'] = valid_col

In [ ]:
del input_gdf
gc.collect()

### CNRA Polygon

In [ ]:
# check if input layer geometry types are correct
try:
    assert input_layers_dict.loc[cnra_polygon_layer_name, 'geometry_type'] in ['Polygon', 'MultiPolygon']
except:
    error_count += 1
    warnings.warn("Input geometry type incorrect")
    error_msg += "Input geometry type incorrect" + "\n"
    
    print("    Valid geom types: ", ['Polygon', 'MultiPolygon'])
    print("    Input geom types: ", input_layers_dict.loc[cnra_polygon_layer_name, 'geometry_type'])
    error_msg += "    Valid geom types: " + str(['Polygon', 'MultiPolygon']) + "\n"
    error_msg += "    Input geom types: " + input_layers_dict.loc[cnra_polygon_layer_name, 'geometry_type'] + "\n"    

In [ ]:
# load data to gdf
input_gdf = gpd.read_file(cnra_input_gdb_path,
                         driver='OpenFileGDB',
                         layer=cnra_polygon_layer_name)

In [ ]:
# schema check
try:
    assert len(set(CNRA_COLUMNS) - set(input_gdf.columns)) == 0
except:
    error_count += 1
    warnings.warn("Not all required fields are present in input")
    error_msg += "Not all required fields are present in input" + "\n"
    
    print("    Missing required fields: ", set(CNRA_COLUMNS) - set(input_gdf.columns))
    error_msg += "    Missing required fields: "+ str(set(CNRA_COLUMNS) - set(input_gdf.columns)) + "\n"

In [ ]:
# uid check
try:
    assert len(input_gdf.globalid.unique()) == len(input_gdf.globalid)
except:
    error_count += 1

    warnings.warn("Exist duplicate unique ID")
    error_msg += "Exist duplicate unique ID" + "\n"
    
    display(input_gdf[input_gdf.duplicated(subset=['globalid'])])
    error_msg += str(list(input_gdf.loc[input_gdf.duplicated(subset=['globalid']), 'globalid'])) + "\n"

In [ ]:
# invalid geometry check
valid_col = input_gdf.is_valid.apply(lambda x: "VALID" if x else "REPAIRED")
try:
    assert sum(input_gdf.is_valid) == len(input_gdf)
except:
    error_count += 1
    warnings.warn('Not all input geometries are valid')
    error_msg += "Not all input geometries are valid" + "\n"
    print("Attempt to repair geometry: ")
    repaired_geom = input_gdf.make_valid()
    # not fixable geometries
    # in pipeline these will be reduced to Point
    print(f"Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries")
    error_msg += f"Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries" + "\n"
    repaired_geom.loc[repaired_geom.geom_type == 'GeometryCollection'] == input_gdf.geometry.loc[repaired_geom.geom_type == 'GeometryCollection']
    valid_col.loc[repaired_geom.geom_type == 'GeometryCollection'] == "INVALID"
    
input_gdf['ValidGeometry'] = valid_col

In [ ]:
del input_gdf
gc.collect()

### CNRA Project

In [16]:
cnra_projects = gpd.read_file(cnra_input_gdb_path, 
                                driver="OpenFileGDB",
                                layer=cnra_project_polygon_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [17]:
# schema check
try:
    assert len(set(CNRA_PROJECT_COLUMNS) - set(cnra_projects.columns)) == 0
except:
    error_count += 1
    warnings.warn("Not all required fields are present in input")
    error_msg += "Not all required fields are present in input" + "\n"
    
    print("    Missing required fields: ", set(CNRA_PROJECT_COLUMNS) - set(cnra_projects.columns))
    error_msg += "    Missing required fields: "+ str(set(CNRA_PROJECT_COLUMNS) - set(cnra_projects.columns)) + "\n"

In [20]:
# uid check
try:
    assert len(cnra_projects.globalid.unique()) == len(cnra_projects.globalid)
except:
    error_count += 1

    warnings.warn("Exist duplicate unique ID")
    error_msg += "Exist duplicate unique ID" + "\n"
    
    display(cnra_projects[cnra_projects.duplicated(subset=['globalid'])])
    error_msg += str(list(cnra_projects.loc[cnra_projects.duplicated(subset=['globalid']), 'globalid'])) + "\n"

In [21]:
print(error_msg)

### CNRA Activity

In [22]:
cnra_activities = gpd.read_file(cnra_input_gdb_path, 
                                driver="OpenFileGDB",
                                layer=cnra_activity_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [23]:
# schema check
try:
    assert len(set(CNRA_ACTIVITY_COLUMNS) - set(cnra_activities.columns)) == 0
except:
    error_count += 1
    warnings.warn("Not all required fields are present in input")
    error_msg += "Not all required fields are present in input" + "\n"
    
    print("    Missing required fields: ", set(CNRA_ACTIVITY_COLUMNS) - set(cnra_activities.columns))
    error_msg += "    Missing required fields: "+ str(set(CNRA_ACTIVITY_COLUMNS) - set(cnra_activities.columns)) + "\n"

In [24]:
# uid check
try:
    assert len(cnra_activities.globalid.unique()) == len(cnra_activities.globalid)
except:
    error_count += 1

    warnings.warn("Exist duplicate unique ID")
    error_msg += "Exist duplicate unique ID" + "\n"
    
    display(cnra_activities[cnra_activities.duplicated(subset=['globalid'])])
    error_msg += str(list(cnra_activities.loc[cnra_activities.duplicated(subset=['globalid']), 'globalid'])) + "\n"

In [25]:
# Empty ACTIVITY_END check
try:
    assert sum(cnra_activities.activity_end.isna()) == 0
except:
    error_count += 1
    
    warnings.warn("Not all data have valid activity end")
    error_msg += "Not all data have valid activity end" + "\n"
    
    print("    NA activity end data:")
    display(cnra_activities[cnra_activities.activity_end.isna()])
    
    # export to excel 
    error_msg += "    NA activity end data UID:" + "\n"
    error_msg += str(list(cnra_activities.loc[cnra_activities.activity_end.isna(), 'globalid'])) + "\n"

    NA activity end data:


C:\Users\sky\AppData\Local\Temp\ipykernel_74748\3401670117.py:7: UserWarning: Not all data have valid activity end
  warnings.warn("Not all data have valid activity end")


,globalid,trmt_geom,activid_user,trmtid_user,treatmentid_poly,treatmentid_pt,treatmentid_ln,org_admin_a,primary_objective,activity_description,activity_cat,counts_to_mas,broad_vegetation_type,bvt_userd,activity_status,activity_quantity,activity_uom,activity_start,activity_end,admin_org_name,implem_org_name,primary_fund_src_name,primary_fund_org_name,secondary_fund_src_name,secondary_fund_org_name,tertiary_fund_src_name,tertiary_fund_org_name,activity_prct,residue_fate,residue_fate_quantity,residue_fate_units,activity_name,created_user,created_date,last_edited_user,last_edited_date
1170,{87970445-D949-4B84-9D1E-C7AB09FC5B3B},Polygon,LRPC EAST COUNTY,LRPC,{6DD46DDA-2D39-4609-81FC-090227F68D45},None,None,SDRC,None,None,None,None,None,None,None,NaN,None,NaT,NaT,SDRC,None,None,SDRC,None,None,None,None,NaN,None,NaN,None,None,cnra.tracker2.sdrc,2024-08-28 19:03:33+00:00,cnra.tracker2.sdrc,2024-08-28 19:03:33+00:00
1171,{01F3A07D-64B8-4653-A8FD-5A58E0F5D1A1},Polygon,LRPC EAST COUNTY,LRPC,{6DD46DDA-2D39-4609-81FC-090227F68D45},None,None,SDRC,None,None,None,None,None,None,None,NaN,None,NaT,NaT,SDRC,None,None,SDRC,None,None,None,None,NaN,None,NaN,None,None,cnra.tracker2.sdrc,2024-08-28 19:07:06+00:00,cnra.tracker2.sdrc,2024-08-28 19:07:06+00:00


In [26]:
# percent of data having activity status as COMPLETE
percent_complete = sum(cnra_activities.activity_status == 'COMPLETE')/len(cnra_activities)
try:
    assert percent_complete > COMPLETE_THRESH
except:
    error_count += 1
    
    warnings.warn("Data does not pass activity status complete threshold")
    error_msg += "Data does not pass activity status complete threshold" + "\n"
    
    print(f"    {percent_complete}% of activity is COMPLETE")
    error_msg += f"    {percent_complete}% of activity is COMPLETE" + "\n"

In [27]:
# 0 activity quantity check
try:
    assert sum(cnra_activities.activity_quantity <= 0) == 0
except:
    error_count += 1
    
    warnings.warn("Exist ACTIVITY_QUANTITY = 0 case")
    error_msg += "Exist ACTIVITY_QUANTITY = 0 case" + "\n"
    print("    0 activity quantity data:")
    display(cnra_activities[cnra_activities.activity_quantity <= 0])
    
    # export to excel 
    error_msg += "    0 activity quantity data:" + "\n"
    error_msg += str(list(cnra_activities[cnra_activities.activity_quantity <= 0].globalid)) + "\n"


    0 activity quantity data:


C:\Users\sky\AppData\Local\Temp\ipykernel_74748\1778320418.py:7: UserWarning: Exist ACTIVITY_QUANTITY = 0 case
  warnings.warn("Exist ACTIVITY_QUANTITY = 0 case")


,globalid,trmt_geom,activid_user,trmtid_user,treatmentid_poly,treatmentid_pt,treatmentid_ln,org_admin_a,primary_objective,activity_description,activity_cat,counts_to_mas,broad_vegetation_type,bvt_userd,activity_status,activity_quantity,activity_uom,activity_start,activity_end,admin_org_name,implem_org_name,primary_fund_src_name,primary_fund_org_name,secondary_fund_src_name,secondary_fund_org_name,tertiary_fund_src_name,tertiary_fund_org_name,activity_prct,residue_fate,residue_fate_quantity,residue_fate_units,activity_name,created_user,created_date,last_edited_user,last_edited_date
115,{FCFFBAC1-D605-4067-AA7F-4B04704F25BC},POLYGON,61147,50110,{3F04420F-77CF-4DC1-9861-5C7E65BD37FC},None,None,RMC,OTHER_FUELS_REDUCTION,MOWING,MECH_HFR,YES,FOREST,NO,ACTIVE,0.0,AC,2022-07-02 00:00:00+00:00,2022-10-30 00:00:00+00:00,RMC,Puente Hills Habitat Preservation Authority,GENERAL_FUND_SB85_2021,RMC,None,None,None,None,NaN,None,NaN,None,LH Fuel Mod Zone 2 B Mowed,reis.thomas@fire.ca.gov,2024-01-12 17:33:23+00:00,mami.odaya@fire.ca.gov,2024-07-11 23:41:19+00:00
116,{575E0C6A-95D6-4728-A867-17D08F07E1BB},POLYGON,61148,50111,{A0F0E1E5-CB0F-44BB-929E-FF18D1C23C96},None,None,RMC,OTHER_FUELS_REDUCTION,MOWING,MECH_HFR,YES,FOREST,NO,ACTIVE,0.0,AC,2022-07-02 00:00:00+00:00,2022-10-30 00:00:00+00:00,RMC,Puente Hills Habitat Preservation Authority,GENERAL_FUND_SB85_2021,RMC,None,None,None,None,NaN,None,NaN,None,LH Fuel Mod Zone 2 B Mowed,reis.thomas@fire.ca.gov,2024-01-12 17:33:23+00:00,mami.odaya@fire.ca.gov,2024-07-11 23:41:19+00:00
6367,{8106A7ED-39D2-4DC0-B477-96FF86AB938E},POLYGON,None,{3071fa19-9b0d-4676-87c7-77c4987a06ba},None,None,None,PARKS,OTHER_FUELS_REDUCTION,PILE_BURN,MECH_HFR,YES,FOREST,YES,COMPLETE,0.0,AC,2024-02-19 00:00:00+00:00,2024-03-29 00:00:00+00:00,PARKS,PARKS,GENERAL_FUND_SB170_2021,PARKS,None,None,None,None,NaN,PILE_BURNING,NaN,None,None,mami.odaya@fire.ca.gov,2024-10-03 16:03:20+00:00,mami.odaya@fire.ca.gov,2024-10-03 16:20:48+00:00
13606,{4E4BF4C9-F410-491D-9DB5-65B676C129A7},POLYGON,CF_33342-454144,CF_16245,{3886152F-6629-4348-937F-096F783453CD},None,None,CALFIRE,OTHER_FUELS_REDUCTION,PILING,MECH_HFR,YES,FOREST,YES,COMPLETE,0.0,AC,2024-05-23 00:00:00+00:00,2024-05-25 00:00:00+00:00,CALFIRE,CALFIRE,GENERAL_FUND,CALFIRE,None,None,None,None,NaN,None,0.0,AC,2024 Piling,mami.odaya@fire.ca.gov,2025-06-20 23:33:37+00:00,mami.odaya@fire.ca.gov,2025-06-21 00:34:35+00:00
13646,{0FFF6BEC-348D-4E84-BC67-6A6B744409B6},POLYGON,CF_29897-412549,CF_14970,{1BDBC5A1-54DA-42F9-952A-078B1E8446DA},None,None,CALFIRE,PRESCRB_FIRE,BROADCAST_BURN,BENEFICIAL_FIRE,YES,FOREST,YES,COMPLETE,0.0,AC,2023-12-31 00:00:00+00:00,2023-12-31 00:00:00+00:00,CALFIRE,CALFIRE,GENERAL_FUND,CALFIRE,None,None,None,None,NaN,BROADCAST_BURN,0.0,AC,2023 - Broadcast Burn,mami.odaya@fire.ca.gov,2025-06-20 23:33:39+00:00,mami.odaya@fire.ca.gov,2025-06-21 00:34:35+00:00
14053,{CC5489E9-4F88-4AAA-A6AE-1338925529D9},POLYGON,CF_32115-430126,CF_15759,{4DCB0E4F-CDA0-4FAC-9CC0-DEF551FD5280},None,None,CALFIRE,OTHER_FUELS_REDUCTION,CHIPPING,MECH_HFR,YES,SHRB_CHAP,YES,COMPLETE,0.0,AC,2024-02-15 00:00:00+00:00,2024-02-15 00:00:00+00:00,CALFIRE,CALFIRE,GENERAL_FUND,CALFIRE,None,None,None,None,NaN,CHIPPING,0.0,AC,2024 Chipping,mami.odaya@fire.ca.gov,2025-06-20 23:34:00+00:00,mami.odaya@fire.ca.gov,2025-06-21 00:34:35+00:00
14123,{3345FEA4-8CC5-4BE7-9711-A54D9E6584B9},POLYGON,CF_31269-407413,CF_15719,{D11329E8-1666-4091-9794-1534C71F607D},None,None,CALFIRE,PRESCRB_FIRE,BROADCAST_BURN,BENEFICIAL_FIRE,YES,GRASS_HERB,YES,COMPLETE,0.0,AC,2023-09-27 00:00:00+00:00,2023-09-27 00:00:00+00:00,CALFIRE,CALFIRE,GENERAL_FUND,CALFIRE,None,None,None,None,NaN,BROADCAST_BURN,0.0,AC,2023 La Panza Ranch Broadcast Burn,mami.odaya@fire.ca.gov,2025-06-20 23:34:03+00:00,mami.odaya@fire.ca.gov,2025-06-21 00:34:35+00:00
14575,{634E46B7-1CC6-4726-B408-DA27BD06CF5C},POLYGON,CF_29574-368694,CF_14845,{7F7B2B16-10FE-423E-843F-F30759E61472},None,None,CALFIRE,OTHER_FUELS_REDUCTION,PILING,MECH_HFR,YES,FOREST,YES,COMPLETE,0

In [28]:
print(error_msg)

Not all data have valid activity end
    NA activity end data UID:
['{87970445-D949-4B84-9D1E-C7AB09FC5B3B}', '{01F3A07D-64B8-4653-A8FD-5A58E0F5D1A1}']
Exist ACTIVITY_QUANTITY = 0 case
    more than 10 rows of 0 activity quantity data exists
    please check the actual code block for full dataframe display



## USFS

In [6]:
usfs_input_base_path = config_inputs['sources']['usfs']['input']['base_path']
region_ids = config_inputs['sources']['usfs']['input']['regions']

In [7]:
# check file type
valid_file_types = ['gdb']
try: 
    
    for region_id in region_ids: 
        usfs_input_file_name = config_inputs['sources']['usfs']['input']['gdb_template'].format(**{'region': region_id})
        usfs_input_gdb_path = os.path.join(usfs_input_base_path, usfs_input_file_name)
        assert usfs_input_gdb_path[-3:] in valid_file_types
except: 
    error_count += 1
    warnings.warn("USFS input is not the correct file type") 
    error_msg += "USFS input is not the correct file type" + "\n"

    print("    Required file type: ", valid_file_types)
    print("    Input file type: ", usfs_input_gdb_path[-3:])
    error_msg += "    Required file type: " + str(valid_file_types) + "\n"
    error_msg += "    Input file type: " + str(usfs_input_gdb_path[-3:]) + "\n"

In [8]:
# check layer name
for region_id in region_ids: 
    usfs_input_file_name = config_inputs['sources']['usfs']['input']['gdb_template'].format(**{'region': region_id})
    usfs_input_gdb_path = os.path.join(usfs_input_base_path, usfs_input_file_name)

    input_layers_dict = gpd.list_layers(usfs_input_gdb_path).set_index('name')
    usfs_input_layer_name = config_inputs['sources']['usfs']['input']['layer_name']
    try: 
        assert usfs_input_layer_name == input_layers_dict.index[0]
    except:
        error_count += 1
        warnings.warn("Not all input layers exist")
        error_msg += "Not all input layers exist" + "\n"

        print("    Provided layer names: ", usfs_input_layer_name)
        print("    Existing layer names: ", list(input_layers_dict.index))
        error_msg += "    Provided layer names: " + str(usfs_input_layer_name) + "\n"
        error_msg += "    Existing layer names: " + str(input_layers_dict.index) + "\n"

In [9]:
# check if input layer geometry types are correct
for region_id in region_ids:
    usfs_input_file_name = config_inputs['sources']['usfs']['input']['gdb_template'].format(**{'region': region_id})
    usfs_input_gdb_path = os.path.join(usfs_input_base_path, usfs_input_file_name)

    input_layers_dict = gpd.list_layers(usfs_input_gdb_path).set_index('name')
    usfs_input_layer_name = config_inputs['sources']['usfs']['input']['layer_name']
    try:
        assert input_layers_dict.loc[usfs_input_layer_name, 'geometry_type'] in ['Polygon', 'MultiPolygon']
    except:
        error_count += 1
        warnings.warn("Input geometry type incorrect")
        error_msg += "Input geometry type incorrect" + "\n"

        print("    Valid geom types: ", ['Polygon', 'MultiPolygon'])
        print("    Input geom types: ", input_layers_dict.loc[usfs_input_layer_name, 'geometry_type'])
        error_msg += "    Valid geom types: " + str(['Polygon', 'MultiPolygon']) + "\n"
        error_msg += "    Input geom types: " + input_layers_dict.loc[usfs_input_layer_name, 'geometry_type'] + "\n"    

In [10]:
USFS_COLUMNS = [
    'SU_SECURITY_ID', 'AU_ORG', 'SUID', 'FACTS_ID', 'SUBUNIT', 'AU_NAME', 'NAME',
    'FEATURE_TYPE', 'SITE_NBR_OF_UNITS', 'GIS_ACRES', 'UOM', 'ACTIVITY_CODE',
    'ACTIVITY', 'LOCAL_QUALIFIER', 'METHOD_CODE', 'METHOD', 'EQUIPMENT_CODE',
    'EQUIPMENT', 'FUND_CODES', 'COST_PER_UNIT', 'WORKFORCE_CODE',
    'FISCAL_YEAR_PLANNED', 'FISCAL_YEAR_AWARDED', 'DATE_AWARDED',
    'FISCAL_YEAR_COMPLETED', 'DATE_COMPLETED', 'NBR_UNITS_PLANNED',
    'NBR_UNITS_ACCOMPLISHED', 'EXCLUDE_ACCOMPLISHMENT', 'TREATMENT_NAME',
    'FUELS_KEYPOINT_AREA', 'ISWUI', 'FIREREGIME', 'CWPP', 'BIOMASS_UTILIZATION',
    'NFPORS_CATEGORY', 'NFPORS_TREATMENT', 'PURPOSE_CODE', 'SALE_NAME', 'SALE_NUMBER',
    'SALE_CATEGORY', 'UNIT_ID', 'PURCHASER_NAME', 'CONTRACT_PLANNED_TERM',
    'AWARD_DATE', 'SALE_CLOSURE_DATE', 'BASE_YEAR', 'KV_NBR_UNITS_FUNDED',
    'PERCENT_FUNDED', 'NEEDS', 'CAUSAL_AGENT', 'REFORESTATION_STATUS', 'EXAM_NBR',
    'NEEDS_ADJUSTMENT', 'TSI', 'EVENT_YEAR', 'IMPLEMENTATION_PROJECT',
    'IMPL_PROJECT_NBR', 'IMPL_PROJECT_TYPE', 'NEPA_DOC_NBR', 'NEPA_DOC_TYPE',
    'NEPA_PROJECT_NAME', 'NEPA_HFI', 'NEPA_HFRA', 'NEPA_SIGNED_DATE',
    'ADMIN_REGION', 'ADMIN_FOREST', 'ADMIN_DISTRICT', 'AU_REGION', 'AU_FOREST',
    'AU_DISTRICT', 'PROC_FOREST', 'OWNERSHIP', 'STATE_ABBR', 'PRODUCTIVITY_CLASS',
    'LAND_SUITABILITY_CODE', 'COUNTY_NAME', 'CONG_DIST_NAME', 'LATITUDE', 'LONGITUDE',
    'LEGAL_LOCATION', 'ASPECT', 'SLOPE', 'ELEVATION', 'WATERSHED_CODE', 'MGT_AREA_CODE',
    'MGT_PRESCRIPTION_CODE', 'ACTIVITY_SITE_REMARKS', 'ACTIVITY_REMARKS',
    'SU_CREATED_BY', 'SU_CREATED_DATE', 'SU_MODIFIED_BY', 'SU_MODIFIED_DATE',
    'ACT_CREATED_BY', 'ACT_CREATED_DATE', 'ACT_MODIFIED_BY', 'ACT_MODIFIED_DATE',
    'ACTIVITY_UNIT_CN', 'LU_CN', 'SUID_CN', 'EVENT_CN', 'NEPA_PROJECT_CN',
    'PALS_PROJECT_CN', 'SALE_CN', 'IMPLEMENTATION_PROJECT_CN', 'UKCN', 'FS_UNIT_ID',
    'CRC_VALUE', 'EVENT_NAME', 'SHAPE_Length', 'SHAPE_Area', 'geometry']

In [58]:
for region_id in region_ids:
    usfs_input_file_name = config_inputs['sources']['usfs']['input']['gdb_template'].format(**{'region': region_id})
    usfs_input_gdb_path = os.path.join(usfs_input_base_path, usfs_input_file_name)
    usfs_input_layer_name = config_inputs['sources']['usfs']['input']['layer_name']
    
    # load data to gdf
    input_gdf = gpd.read_file(usfs_input_gdb_path,
                             driver='OpenFileGDB',
                             layer=usfs_input_layer_name)
    
    # schema check
    try:
        assert len(set(USFS_COLUMNS) - set(input_gdf.columns)) == 0
    except:
        error_count += 1
        warnings.warn("Not all required fields are present in input")
        error_msg += "Not all required fields are present in input" + "\n"

        print("    Missing required fields: ", set(USFS_COLUMNS) - set(input_gdf.columns))
        error_msg += "    Missing required fields: "+ str(set(USFS_COLUMNS) - set(input_gdf.columns)) + "\n"
        
        
    # uid check
    try:
        assert len(input_gdf[input_gdf.duplicated(subset=["NEPA_DOC_NBR","SUID"])]) == 0
    except:
        error_count += 1

        warnings.warn("Exist duplicate unique ID")
        error_msg += "Exist duplicate unique ID" + "\n"

        display(input_gdf[input_gdf.duplicated(subset=["NEPA_DOC_NBR","SUID"])]) 
        
    # invalid geometry check
    valid_col = input_gdf.is_valid.apply(lambda x: "VALID" if x else "REPAIRED")
    try:
        assert sum(input_gdf.is_valid) == len(input_gdf)
    except:
        error_count += 1
        warnings.warn('Not all input geometries are valid')
        error_msg += "Not all input geometries are valid" + "\n"
        print("Attempt to repair geometry: ")
        repaired_geom = input_gdf.make_valid()
        # not fixable geometries
        # in pipeline these will be reduced to Point
        print(f"Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries")
        error_msg += f"Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries" + "\n"
        repaired_geom.loc[repaired_geom.geom_type == 'GeometryCollection'] == input_gdf.geometry.loc[repaired_geom.geom_type == 'GeometryCollection']
        valid_col.loc[repaired_geom.geom_type == 'GeometryCollection'] == "INVALID"

    input_gdf['ValidGeometry'] = valid_col
    
    
    # Empty ACTIVITY_END check
    try:
        assert sum(~(input_gdf.DATE_COMPLETED.notna() + input_gdf.NEPA_SIGNED_DATE.notna())) == 0
    except:
        error_count += 1

        warnings.warn("Not all data have valid activity end")
        error_msg += "Not all data have valid activity end" + "\n"

        print("    NA activity end data:")
        display(input_gdf[~(input_gdf.DATE_COMPLETED.notna() + input_gdf.NEPA_SIGNED_DATE.notna())])

        # export to excel 
    
    # percent of data having activity status as COMPLETE
    percent_complete = sum(input_gdf.DATE_COMPLETED.notna())/len(input_gdf)
    try:
        assert percent_complete > COMPLETE_THRESH
    except:
        error_count += 1

        warnings.warn("Data does not pass activity status complete threshold")
        error_msg += "Data does not pass activity status complete threshold" + "\n"

        print(f"    {percent_complete}% of activity is COMPLETE")
        error_msg += f"    {percent_complete}% of activity is COMPLETE" + "\n"
        
        
    # 0 activity quantity check
    try:
        assert sum(input_gdf['NBR_UNITS_ACCOMPLISHED'].fillna(input_gdf['NBR_UNITS_PLANNED']) <= 0) == 0
    except:
        error_count += 1

        warnings.warn("Exist ACTIVITY_QUANTITY = 0 case")
        error_msg += "Exist ACTIVITY_QUANTITY = 0 case" + "\n"
        print("    0 activity quantity data:")
        display(input_gdf[input_gdf['NBR_UNITS_ACCOMPLISHED'].fillna(input_gdf['NBR_UNITS_PLANNED']) <= 0])

        # export to excel 

    del input_gdf
    gc.collect()

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


KeyboardInterrupt: 

## IFPRS

In [6]:
ifprs_input_gdb_path = config_inputs['sources']['ifprs']['input']['gdb_path']
ifprs_input_layer_name = config_inputs['sources']['ifprs']['input']['layer_name']

In [7]:
IFPRS_COLUMNS = [
    'Name', 'Agency', 'CalculatedArea', 'IsWUI', 'Shape_Area', 'Status', 'InitiationDate', 'CompletionDate',
    'TotalAcres', 'Type', 'SubType', 'Notes', 'CompletionFiscalYear', 'County', 'EntityID'
]

In [8]:
# check file type
valid_file_types = ['gdb']
try: 
    assert ifprs_input_gdb_path[-3:] in valid_file_types
except: 
    error_count += 1
    warnings.warn("IFPRS input is not the correct file type") 
    error_msg += "IFPRS input is not the correct file type" + "\n"

    print("    Required file type: ", valid_file_types)
    print("    Input file type: ", ifprs_input_gdb_path[-3:])
    error_msg += "    Required file type: " + str(valid_file_types) + "\n"
    error_msg += "    Input file type: " + str(ifprs_input_gdb_path[-3:]) + "\n"

In [9]:
# check input layers
input_layers_dict = gpd.list_layers(ifprs_input_gdb_path).set_index('name')
# check if all input layer names are correct, exist in input gdb file
try:
    assert ifprs_input_layer_name in set(input_layers_dict.index)
except:
    error_count += 1
    warnings.warn("Not all input layers exist")
    error_msg += "Not all input layers exist" + "\n"
    
    print("    Existing layer names: ", list(input_layers_dict.index))
    error_msg += "    Provided layer names: " + str(ifprs_input_layer_name) + "\n"
    error_msg += "    Existing layer names: " + str(input_layers_dict.index) + "\n"

In [10]:
# check if input layer geometry types are correct
try:
    assert input_layers_dict.loc[ifprs_input_layer_name, 'geometry_type'] in ['Polygon', 'MultiPolygon']
except:
    error_count += 1
    warnings.warn("Input geometry type incorrect")
    error_msg += "Input geometry type incorrect" + "\n"
    
    print("    Valid geom types: ", ['Polygon', 'MultiPolygon'])
    print("    Input geom types: ", input_layers_dict.loc[ifprs_input_layer_name, 'geometry_type'])
    error_msg += "    Valid geom types: " + str(['Polygon', 'MultiPolygon']) + "\n"
    error_msg += "    Input geom types: " + input_layers_dict.loc[ifprs_input_layer_name, 'geometry_type'] + "\n"    

In [11]:
# load data to gdf
input_gdf = gpd.read_file(ifprs_input_gdb_path,
                         driver='OpenFileGDB',
                         layer=ifprs_input_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [16]:
input_gdf

,ID,JoinID,JoinFeature,EntityID,EntityType,EntityCategory,IsPoint,ParentID,ParentObjectID,IsParentTreatment,...,EstimateJoinFeature,ActualJoinID,ActualJoinFeature,IsReadOnly,IsInteragency,InteragencyName,AreaUnit,Shape_Length,Shape_Area,geometry
0,19,19,Polygon,aa684c90-7517-47fd-87a0-29154559d1ad,Fuels,None,No,None,None,None,...,Polygon,19.0,Polygon,None,None,None,Acres,0.020184,0.000021,"MULTIPOLYGON (((-105.01646 35.90945, -105.0158..."
1,21,21,Polygon,4eaaab16-9fb4-4b9f-85cd-d533b33cb4c4,Fuels,None,No,None,None,None,...,Polygon,21.0,Polygon,None,None,None,Acres,0.044140,0.000010,"MULTIPOLYGON (((-112.25097 43.97961, -112.2495..."
2,22,22,Polygon,None,Fuels,None,No,None,None,None,...,Polygon,22.0,Polygon,None,None,None,Acres,0.010049,0.000004,"MULTIPOLYGON (((-68.32487 44.42267, -68.32476 ..."
3,28,28,Polygon,9f7414b6-5d17-46d4-bc31-5ee518e73567,Fuels,None,No,None,None,None,...,Polygon,28.0,Polygon,None,None,None,Acres,0.031223,0.000015,"MULTIPOLYGON (((-110.4077 44.55269, -110.4062 ..."
4,31,31,Polygon,1d04b5a1-e07b-4758-af30-752ec94084f8,Fuels,None,No,None,None,None,...,Polygon,31.0,Polygon,None,None,None,Acres,0.059326,0.000020,"MULTIPOLYGON (((-99.72325 48.86304, -99.72205 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4783,46412,45951,Polygon,b55c909e-c1f6-426f-a9c4-038ca690256f,Fuels,None,No,None,None,None,...,Polygon,45951.0,Polygon,None,None,None,Acres,0.137984,0.000631,"MULTIPOLYGON (((-121.46854 44.80287, -121.4685..."
4784,46414,45953,Polygon,fea200a6-b7d5-4cb1-b361-48c1a1895ee9,Fuels,None,No,None,None,None,...,Polygon,45953.0,Polygon,None,None,None,Acres,6.325954,0.000224,"MULTIPOLYGON (((-102.38947 43.58916, -102.3892..."
4785,46449,45988,Polygon,021f5946-0b92-4009-afc8-4b122e9569e7,Fuels,None,No,None,None,None,...,Polygon,45988.0,Polygon,None,None,None,Acres,0.082766,0.000050,"MULTIPOLYGON (((-98.25528 42.99173, -98.2553 4..."
4786,46451,45990,Polygon,0ebcea6b-3a8d-4620-9ba9-3963913e5bfa,Fuels,None,No,None,None,None,...,Polygon,45990.0,Polygon,None,None,None,Acres,0.149291,0.000178,"MULTIPOLYGON (((-118.02582 47.93329, -118.0255..."


In [23]:
# schema check
try:
    assert len(set(IFPRS_COLUMNS) - set(input_gdf.columns)) == 0
except:
    error_count += 1
    warnings.warn("Not all required fields are present in input")
    error_msg += "Not all required fields are present in input" + "\n"
    
    print("    Missing required fields: ", set(IFPRS_COLUMNS) - set(input_gdf.columns))
    error_msg += "    Missing required fields: "+ str(set(IFPRS_COLUMNS) - set(input_gdf.columns)) + "\n"

In [17]:
# uid check
try:
    #assert len(input_gdf.GlobalID.unique()) == len(input_gdf)
    assert len(input_gdf.ID.unique()) == len(input_gdf)
    
except:
    error_count += 1

    warnings.warn("Exist duplicate unique ID")
    error_msg += "Exist duplicate unique ID" + "\n"
    
    display(input_gdf[input_gdf.duplicated(subset=['ID'])])
    error_msg += str(list(input_gdf.loc[input_gdf.duplicated(subset=['ID']), 'ID'])) + "\n"

In [65]:
# invalid geometry check
valid_col = input_gdf.is_valid.apply(lambda x: "VALID" if x else "REPAIRED")
try:
    assert sum(input_gdf.is_valid) == len(input_gdf)
except:
    error_count += 1
    warnings.warn('Not all input geometries are valid')
    error_msg += "Not all input geometries are valid" + "\n"
    print("Attempt to repair geometry: ")
    repaired_geom = input_gdf.make_valid()
    # not fixable geometries
    # in pipeline these will be reduced to Point
    print(f"Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries")
    error_msg += f"Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries" + "\n"
    repaired_geom.loc[repaired_geom.geom_type == 'GeometryCollection'] == input_gdf.geometry.loc[repaired_geom.geom_type == 'GeometryCollection']
    valid_col.loc[repaired_geom.geom_type == 'GeometryCollection'] == "INVALID"
    
input_gdf['ValidGeometry'] = valid_col

C:\Users\sky\AppData\Local\Temp\ipykernel_86948\3967708979.py:7: UserWarning: Not all input geometries are valid
  warnings.warn('Not all input geometries are valid')


Attempt to repair geometry: 
Found 0 unrepairable geometries


In [72]:
# Empty ACTIVITY_END check
try:
    assert sum(input_gdf.Completion.isna()) == 0
except:
    error_count += 1
    
    warnings.warn("Not all data have valid activity end")
    error_msg += "Not all data have valid activity end" + "\n"
    
    print("    NA activity end data:")
    display(input_gdf[input_gdf.Completion.isna()])
    
    # export to excel 

    NA activity end data:


C:\Users\sky\AppData\Local\Temp\ipykernel_86948\318226974.py:7: UserWarning: Not all data have valid activity end
  warnings.warn("Not all data have valid activity end")


,ActualTrea,IsPoint,CreatedBy,Name,Unit,Region,Agency,Department,IsDepartme,Latitude,...,IsFundingA,ID,OldFCID,OldOBJECTI,Shape__Are,Shape__Len,Shape_Length,Shape_Area,geometry,ValidGeometry
11,7eecc82b-ac5a-4db7-808d-598c60116294,No,jadams_nifc,TXSNR-HFR-FY24-RX,San Bernard National Wildlife Refuge,Southwest Region,FWS,DOI,No,28.866835,...,,0,7eecc82b-ac5a-4db7-808d-598c60116294,4402,0.000786,0.134202,0.134202,0.000786,"MULTIPOLYGON (((-95.57361 28.8415, -95.57386 2...",VALID
15,a39d471b-9e5a-45f2-a050-4169b728a20c,No,trankin_nifc,WATBR-BIL-FY25-RX-Broadcast Area,Turnbull National Wildlife Refuge,Pacific Region,FWS,DOI,No,47.421259,...,,0,a39d471b-9e5a-45f2-a050-4169b728a20c,5602,0.000174,0.063359,0.063359,0.000174,"MULTIPOLYGON (((-117.62856 47.41678, -117.6257...",VALID
18,b9ee7925-bad7-47d4-a5b4-4733d6759969,No,jhickerson_nifc,Five Lakes FY24 Prep - BIL,Chickasaw National Recreation Area,Intermountain,NPS,Department of the Interior,No,34.467115,...,,0,b9ee7925-bad7-47d4-a5b4-4733d6759969,6402,0.000021,0.212274,0.212274,0.000021,"MULTIPOLYGON (((-96.99446 34.441, -96.99476 34...",VALID
21,4a02ddff-9197-4ac2-8992-9b9415943925,No,jhickerson_nifc,West Side FY24 Thin - BIL,Glacier National Park,Intermountain,NPS,Department of the Interior,No,48.533533,...,,0,4a02ddff-9197-4ac2-8992-9b9415943925,7203,0.000006,0.026660,0.026660,0.000006,"MULTIPOLYGON (((-113.99938 48.53168, -113.9994...",VALID
29,56f4ad9a-f267-417b-bbde-49e241ac0e19,No,jhickerson_nifc,Five Lakes FY24 RX - BIL,Chickasaw National Recreation Area,Intermountain,NPS,Department of the Interior,No,34.469221,...,,0,56f4ad9a-f267-417b-bbde-49e241ac0e19,7606,0.001050,0.239214,0.239214,0.001050,"MULTIPOLYGON (((-97.00479 34.47233, -97.00475 ...",VALID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963,7640fd22-6471-4bac-8f23-3ab1778c660d,No,sean.nichols_nifc,Wrights Bottom Rx,Talihina Agency,Eastern Oklahoma,"BIA, Tribal",Department of the Interior,No,34.742335,...,,0,7640fd22-6471-4bac-8f23-3ab1778c660d,336393,0.000059,0.033521,0.033521,0.000059,"MULTIPOLYGON (((-95.12146 34.74358, -95.11939 ...",VALID
1965,7ed497a7-f41a-4b96-922b-b570ef24cc0d,No,sean.nichols_nifc,Kiamichi River Rx,Eastern Regional Office,Eastern Oklahoma,"BIA, Tribal",Department of the Interior,No,34.614545,...,,0,7ed497a7-f41a-4b96-922b-b570ef24cc0d,336796,0.000080,0.044848,0.044848,0.000080,"MULTIPOLYGON (((-95.29875 34.61738, -95.29614 ...",VALID
1976,20b98b77-fb99-487e-afda-8662e717f72f,No,benjamin.marquez_nifc,Road Side Rx-Burn,N/A,N/A,N/A,N/A,Yes,33.704024,...,,0,20b98b77-fb99-487e-afda-8662e717f72f,341203,0.000210,0.085391,0.085391,0.000210,"MULTIPOLYGON (((-110.52939 33.70415, -110.5298...",VALID
1979,de5b3c34-4303-496c-b9e7-d139dd483e95,No,christinethompson_nifc,Malay Gap Prescribed Burn,San Carlos Agency,Western,"BIA, Tribal",Department of the Interior,No,33.583112,...,No,0,de5b3c34-4303-496c-b9e7-d139dd483e95,342418,0.000387,0.137573,0.137573,0.000387,"MULTIPOLYGON (((-109.50416 33.58465, -109.5032...",VALID


In [67]:
# percent of data having activity status as COMPLETE
percent_complete = sum(input_gdf.Status == 'Completed')/len(input_gdf)
try:
    assert percent_complete > COMPLETE_THRESH
except:
    error_count += 1
    
    warnings.warn("Data does not pass activity status complete threshold")
    error_msg += "Data does not pass activity status complete threshold" + "\n"
    
    print(f"    {percent_complete}% of activity is COMPLETE")
    error_msg += f"    {percent_complete}% of activity is COMPLETE" + "\n"

In [74]:
# 0 activity quantity check
try:
    assert sum(input_gdf.TotalAcres <= 0) == 0
except:
    error_count += 1
    
    warnings.warn("Exist ACTIVITY_QUANTITY = 0 case")
    error_msg += "Exist ACTIVITY_QUANTITY = 0 case" + "\n"
    print("    0 activity quantity data:")
    display(input_gdf[input_gdf.TotalAcres <= 0])

    0 activity quantity data:


C:\Users\sky\AppData\Local\Temp\ipykernel_86948\2207213422.py:7: UserWarning: Exist ACTIVITY_QUANTITY = 0 case
  warnings.warn("Exist ACTIVITY_QUANTITY = 0 case")


,ActualTrea,IsPoint,CreatedBy,Name,Unit,Region,Agency,Department,IsDepartme,Latitude,...,IsFundingA,ID,OldFCID,OldOBJECTI,Shape__Are,Shape__Len,Shape_Length,Shape_Area,geometry,ValidGeometry
0,803a6731-0588-40dc-870e-c57a2803c4b8,No,jwagner_nifc,NDAWR-HFR-FY24-HFR-RX-Arrowwood Refuge,Arrowwood National Wildlife Refuge,Mountain Prairie Region,FWS,Department of the Interior,No,47.203030,...,,0,803a6731-0588-40dc-870e-c57a2803c4b8,2002,2.786232e-04,0.115465,0.115465,2.786232e-04,"MULTIPOLYGON (((-98.82746 47.2042, -98.82746 4...",VALID
1,164ef5f9-c4a8-4952-a8f5-809127822e0b,No,cory_thompson_nifc,WACNR-HFR-FY24-RX-OXBOW,Conboy Lake National Wildlife Refuge,Pacific Region,FWS,DOI,No,45.988317,...,,0,164ef5f9-c4a8-4952-a8f5-809127822e0b,2402,3.531115e-05,0.031194,0.031194,3.531115e-05,"MULTIPOLYGON (((-121.257 45.99479, -121.25599 ...",VALID
2,868651c9-5011-470d-a991-e467912ad6dc,No,cory_thompson_nifc,WACNR-HFR-FY24-MX-Lodgepole Piles Kelly Unit,Conboy Lake National Wildlife Refuge,Pacific Region,FWS,DOI,No,46.006968,...,,0,868651c9-5011-470d-a991-e467912ad6dc,2403,2.945598e-06,0.008858,0.008858,2.945598e-06,"MULTIPOLYGON (((-121.24068 46.00825, -121.2395...",VALID
3,857ba39b-fda6-45c1-a58c-0b7447281f53,No,jbrodbeck_nifc,CATKR-HFR-FY24-RX-W,Tule Lake National Wildlife Refuge,Pacific Southwest Region,FWS,Department of the Interior,No,41.841583,...,,0,857ba39b-fda6-45c1-a58c-0b7447281f53,2802,5.718279e-04,0.156313,0.156313,5.718279e-04,"MULTIPOLYGON (((-121.4294 41.84987, -121.42916...",VALID
4,7f528b3d-7369-4283-a6c1-b88cad4585e9,No,kmeinhart_NIFC,WATBR-BIL-FY24-MX-Grogan Thin - copy,Turnbull National Wildlife Refuge,Pacific Region,FWS,DOI,No,47.398975,...,,0,7f528b3d-7369-4283-a6c1-b88cad4585e9,2803,1.208919e-04,0.053311,0.053311,1.208919e-04,"MULTIPOLYGON (((-117.55838 47.40535, -117.5528...",VALID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,42e99122-f785-47c8-b721-448fffc74eec,No,truzicka_nifc,SDMDR_HFR_FY25_RX_Anderson_Piles,Madison Wetland Management District,Mountain Prairie Region,FWS,Department of the Interior,No,44.106916,...,No,0,42e99122-f785-47c8-b721-448fffc74eec,346436,3.152089e-06,0.007684,0.007684,3.152090e-06,"MULTIPOLYGON (((-96.84863 44.1075, -96.84603 4...",VALID
1990,99b0da91-5de0-4c91-8024-0752cf4d30f0,No,truzicka_nifc,SDMDR_HFR_FY25_RX_Payne_PIles,Madison Wetland Management District,Mountain Prairie Region,FWS,Department of the Interior,No,43.977998,...,No,0,99b0da91-5de0-4c91-8024-0752cf4d30f0,346837,1.497939e-07,0.001745,0.001745,1.497940e-07,"MULTIPOLYGON (((-97.07429 43.97824, -97.07394 ...",VALID
1991,7427e56c-ce6c-4d5f-b689-a557cb612073,No,truzicka_nifc,SDMDR_HFR_FY25_RX_Madison_Piles,Madison Wetland Management District,Mountain Prairie Region,FWS,Department of the Interior,No,43.971557,...,No,0,7427e56c-ce6c-4d5f-b689-a557cb612073,347238,2.195616e-06,0.006419,0.006419,2.195617e-06,"MULTIPOLYGON (((-97.07347 43.97256, -97.07231 ...",VALID
1992,83e86e5c-a87b-4772-8bfc-ba929d4733af,No,truzicka_nifc,SDSLR_HFR_FY25_RX_SandLake_Piles,Sand Lake National Wildlife Refuge,Mountain Prairie Region,FWS,Department of the Interior,No,45.730023,...,No,0,83e86e5c-a87b-4772-8bfc-ba929d4733af,347641,1.427464e-07,0.001604,0.001604,1.427463e-07,"MULTIPOLYGON (((-98.29592 45.73028, -98.29562 ...",VALID


In [77]:
del input_gdf
gc.collect()

In [6]:
# flag GIS area is smaller than reported acres

## Timber Nonspatial

In [8]:
tn_input_excel_path = config_inputs['sources']['timber_industry_nonspatial']['input']['excel_path']

In [9]:
TIMBER_NONSPATIAL_COLUMNS = [
    'ACTIVITY CATEGORY', 'ACTIVITY DESCRIPTION', 'ACTIVITY END', 'ACTIVITY QUANTITY',
    'ACTIVITY START', 'ACTIVITY STATUS', 'ACTIVITY UNITS', 'ADMINISTERING ORG',
    'BROAD VEGETATION TYPE', 'COUNTY', 'IN WUI', 'IS BVT USER DEFINED',
    'PRIMARY OWNERSHIP GROUP', 'TASK FORCE REGION'
]

'xlsx'

In [12]:
# check file type
valid_file_types = ['xlsx']
try: 
    assert tn_input_excel_path.split('.')[-1] in valid_file_types
except: 
    error_count += 1
    warnings.warn("Timber Nonspatial input is not the correct file type") 
    error_msg += "Timber Nonspatial input is not the correct file type" + "\n"

    print("    Required file type: ", valid_file_types)
    print("    Input file type: ", tn_input_excel_path.split('.')[-1])
    error_msg += "    Required file type: " + str(valid_file_types) + "\n"
    error_msg += "    Input file type: " + str(tn_input_excel_path.split('.')[-1]) + "\n"

In [13]:
# load data to gdf
input_gdf = pd.read_excel(tn_input_excel_path, )

In [14]:
# schema check
try:
    assert len(set(TIMBER_NONSPATIAL_COLUMNS) - set(input_gdf.columns)) == 0
except:
    error_count += 1
    warnings.warn("Not all required fields are present in input")
    error_msg += "Not all required fields are present in input" + "\n"
    
    print("    Missing required fields: ", set(TIMBER_NONSPATIAL_COLUMNS) - set(input_gdf.columns))
    error_msg += "    Missing required fields: "+ str(set(TIMBER_NONSPATIAL_COLUMNS) - set(input_gdf.columns)) + "\n"

In [17]:
# Empty ACTIVITY_END check
try:
    assert sum(input_gdf['ACTIVITY END'].isna()) == 0
except:
    error_count += 1
    
    warnings.warn("Not all data have valid activity end")
    error_msg += "Not all data have valid activity end" + "\n"
    
    print("    NA activity end data:")
    display(input_gdf[input_gdf.Completion.isna()])
    
    # export to excel 

In [20]:
# percent of data having activity status as COMPLETE
percent_complete = sum(input_gdf['ACTIVITY STATUS'] == 'Complete')/len(input_gdf)
try:
    assert percent_complete > COMPLETE_THRESH
except:
    error_count += 1
    
    warnings.warn("Data does not pass activity status complete threshold")
    error_msg += "Data does not pass activity status complete threshold" + "\n"
    
    print(f"    {percent_complete}% of activity is COMPLETE")
    error_msg += f"    {percent_complete}% of activity is COMPLETE" + "\n"

In [21]:
# 0 activity quantity check
try:
    assert sum(input_gdf['ACTIVITY QUANTITY'] <= 0) == 0
except:
    error_count += 1
    
    warnings.warn("Exist ACTIVITY_QUANTITY = 0 case")
    error_msg += "Exist ACTIVITY_QUANTITY = 0 case" + "\n"
    print("    0 activity quantity data:")
    display(input_gdf[input_gdf['ACTIVITY QUANTITY'] <= 0])

    0 activity quantity data:


C:\Users\sky\AppData\Local\Temp\ipykernel_200020\4099700465.py:7: UserWarning: Exist ACTIVITY_QUANTITY = 0 case
  warnings.warn("Exist ACTIVITY_QUANTITY = 0 case")


,Unnamed: 0,ACTIVITY DESCRIPTION,ACTIVITY CATEGORY,BROAD VEGETATION TYPE,IS BVT USER DEFINED,ACTIVITY STATUS,ACTIVITY QUANTITY,ACTIVITY UNITS,ACTIVITY START,ACTIVITY END,ADMINISTERING ORG,COUNTY,IN WUI,TASK FORCE REGION,PRIMARY OWNERSHIP GROUP
13,13,Fuel Break,Mechanical and Hand Fuels Reduction,Forest,Yes,Complete,0.0,Acres,2021-01-01,2021-12-31,Industrial Timber,Statewide,Non-WUI (user defined),Statewide,Private - Industrial
25,0,Alternative Prescription,Timber Harvest,Forest,Yes,Complete,0.0,Acres,2022-01-01,2022-12-31,Industrial Timber,Statewide,Non-WUI (user defined),Statewide,Private - Industrial
32,7,Sanitation Harvest,Timber Harvest,Forest,Yes,Complete,0.0,Acres,2022-01-01,2022-12-31,Industrial Timber,Statewide,Non-WUI (user defined),Statewide,Private - Industrial
35,10,Site Preparation,Tree Planting,Forest,Yes,Complete,0.0,Acres,2022-01-01,2022-12-31,Industrial Timber,Statewide,Non-WUI (user defined),Statewide,Private - Industrial
40,15,Invasive Plant Removal,Mechanical and Hand Fuels Reduction,Forest,Yes,Complete,0.0,Acres,2022-01-01,2022-12-31,Industrial Timber,Statewide,Non-WUI (user defined),Statewide,Private - Industrial
41,16,Landing Treated,Mechanical and Hand Fuels Reduction,Forest,Yes,Complete,0.0,Acres,2022-01-01,2022-12-31,Industrial Timber,Statewide,Non-WUI (user defined),Statewide,Private - Industrial
47,22,Roadway Clearance,Mechanical and Hand Fuels Reduction,Forest,Yes,Complete,0.0,Acres,2022-01-01,2022-12-31,Industrial Timber,Statewide,Non-WUI (user defined),Statewide,Private - Industrial
49,24,Thinning (Mechanical),Mechanical and Hand Fuels Reduction,Forest,Yes,Complete,0.0,Acres,2022-01-01,2022-12-31,Industrial Timber,Statewide,Non-WUI (user defined),Statewide,Private - Industrial
51,1,Aspen/Meadow/Wet Area Restoration,Timber Harvest,Forest,Yes,Complete,0.0,Acres,2023-01-01,2023-12-31,Industrial Timber,Statewide,Non-WUI (user defined),Statewide,Private - Industrial
58,8,Transition Harvest,Timber Harvest,Forest,Yes,Complete,0.0,Acres,2023-01-01,2023-12-31,Industrial Timber,Statewide,Non-WUI (user defined),Statewide,Private - Industrial


In [22]:
del input_gdf
gc.collect()

3961

## Timber Spatial

In [28]:
ti_input_gdb_path = config_inputs['sources']['timber_industry_spatial']['input']['gdb_path']
ti_input_layer_name = config_inputs['sources']['timber_industry_spatial']['input']['layer_name']

In [29]:
TIMBER_INDUSTRY_SPATIAL_COLUMNS = [
    'Organization', 'Org_Public', 'ID', 'Name', 'Objective', 'Status', 
    'Year', 'BroadVegType', 'GISACRES', 'Shape_Length', 'Shape_Area', 
    'OBJECTID', 'geometry', 
]

In [31]:
# check file type
valid_file_types = ['gdb']
try: 
    assert ti_input_gdb_path[-3:] in valid_file_types
except: 
    error_count += 1
    warnings.warn("Timber Spatial input is not the correct file type") 
    error_msg += "Timber Spatial input is not the correct file type" + "\n"

    print("    Required file type: ", valid_file_types)
    print("    Input file type: ", ti_input_gdb_path[-3:])
    error_msg += "    Required file type: " + str(valid_file_types) + "\n"
    error_msg += "    Input file type: " + str(ti_input_gdb_path[-3:]) + "\n"

In [32]:
# check input layers
input_layers_dict = gpd.list_layers(ti_input_gdb_path).set_index('name')
# check if all input layer names are correct, exist in input gdb file
try:
    assert ti_input_layer_name in set(input_layers_dict.index)
except:
    error_count += 1
    warnings.warn("Not all input layers exist")
    error_msg += "Not all input layers exist" + "\n"
    
    print("    Existing layer names: ", list(input_layers_dict.index))
    error_msg += "    Provided layer names: " + str(ti_input_layer_name) + "\n"
    error_msg += "    Existing layer names: " + str(input_layers_dict.index) + "\n"

In [36]:
# check if input layer geometry types are correct
try:
    assert input_layers_dict.loc[ti_input_layer_name, 'geometry_type'] in ['Polygon', 'MultiPolygon']
except:
    error_count += 1
    warnings.warn("Input geometry type incorrect")
    error_msg += "Input geometry type incorrect" + "\n"
    
    print("    Valid geom types: ", ['Polygon', 'MultiPolygon'])
    print("    Input geom types: ", input_layers_dict.loc[ti_input_layer_name, 'geometry_type'])
    error_msg += "    Valid geom types: " + str(['Polygon', 'MultiPolygon']) + "\n"
    error_msg += "    Input geom types: " + input_layers_dict.loc[ti_input_layer_name, 'geometry_type'] + "\n"    

In [37]:
# load data to gdf
input_gdf = gpd.read_file(ti_input_gdb_path,
                         driver='OpenFileGDB',
                         layer=ti_input_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [38]:
# schema check
try:
    assert len(set(TIMBER_INDUSTRY_SPATIAL_COLUMNS) - set(input_gdf.columns)) == 0
except:
    error_count += 1
    warnings.warn("Not all required fields are present in input")
    error_msg += "Not all required fields are present in input" + "\n"
    
    print("    Missing required fields: ", set(TIMBER_INDUSTRY_SPATIAL_COLUMNS) - set(input_gdf.columns))
    error_msg += "    Missing required fields: "+ str(set(TIMBER_INDUSTRY_SPATIAL_COLUMNS) - set(input_gdf.columns)) + "\n"

    Missing required fields:  {'Organization', 'OBJECTID'}


C:\Users\sky\AppData\Local\Temp\ipykernel_200020\301411796.py:6: UserWarning: Not all required fields are present in input
  warnings.warn("Not all required fields are present in input")


In [40]:
# uid check
try:
    #assert len(input_gdf.GlobalID.unique()) == len(input_gdf)
    assert len(input_gdf.ID.unique()) == len(input_gdf)
    
except:
    error_count += 1

    warnings.warn("Exist duplicate unique ID")
    error_msg += "Exist duplicate unique ID" + "\n"
    
    display(input_gdf[input_gdf.duplicated(subset=['ID'])])
    error_msg += str(list(input_gdf.loc[input_gdf.duplicated(subset=['ID']), 'ID'])) + "\n"

C:\Users\sky\AppData\Local\Temp\ipykernel_200020\3221479284.py:9: UserWarning: Exist duplicate unique ID
  warnings.warn("Exist duplicate unique ID")


,Org_Public,ID,Name,Objective,Shape_Length,Shape_Area,Activity,Status,StartDate,EndDate,BroadVegType,Year,GISACRES,TreatedAcres,geometry
2,Private,19-FH-NEU-012,Yuba Foothills Healthy Forest Project,Fuel Reduction,82074.564178,4.857025e+06,None,Proposed,NaT,NaT,Timber,NaN,1200.196965,NaN,"MULTIPOLYGON (((-100357.557 150779.573, -10034..."
5,Private,2-15EX-112-PLU,Lee Summit 1038j Pilot Exemption,Fuel Reduction,4285.272008,9.020583e+05,None,Exists,NaT,NaT,Timber,2015.0,222.903473,NaN,"MULTIPOLYGON (((-63898.536 207483.598, -63891...."
6,Private,2-15EX-112-PLU,Lee Summit Mastication-Stevens Funds,Fuel Reduction,2812.367368,3.287648e+05,None,Exists,NaT,NaT,Timber,2016.0,81.239553,NaN,"MULTIPOLYGON (((-63511.137 207474.86, -63508.6..."
8,Private,2-15-018-PLU,Icky Thump Fuelbreak,Fuel Break,3850.087687,1.330670e+05,None,Exists,NaT,NaT,Timber,2017.0,32.881574,NaN,"MULTIPOLYGON (((-76065.753 212351.381, -76076...."
15,Private,2-18-062-PLU,Chester Flats THP,Fuel Reduction,10699.231680,2.843457e+06,None,Exists,NaT,NaT,Timber,2020.0,702.632526,NaN,"MULTIPOLYGON (((-106034.593 243659.873, -10586..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,Private,187,FFSC202304187,Fuel Break,1737.592047,6.058304e+04,Herbicide Application,Complete,2024-07-01 00:00:00+00:00,2024-07-14 00:00:00+00:00,Woodland,2024.0,14.970373,NaN,"MULTIPOLYGON (((-31288.663 14307.973, -31301.3..."
821,Private,187,FFSC202304187,Fuel Break,1737.592047,6.058304e+04,Herbicide Application,Complete,2021-12-31 00:00:00+00:00,2021-12-31 00:00:00+00:00,Woodland,2021.0,14.970373,NaN,"MULTIPOLYGON (((-31288.663 14307.973, -31301.3..."
822,Private,203,FFSC202304203,Fuel Break,1216.197581,7.288934e+04,Herbicide Application,Complete,2023-12-31 00:00:00+00:00,2023-12-31 00:00:00+00:00,Woodland,2023.0,18.011320,NaN,"MULTIPOLYGON (((-32409.004 13733.429, -32409.5..."
823,Private,189,FFSC202204189,Fuel Break,934.102148,4.175915e+04,Mastication,Complete,2022-10-14 00:00:00+00:00,2022-10-14 00:00:00+00:00,Timber,2022.0,10.318894,NaN,"MULTIPOLYGON (((-20162.9 4564.545, -20160.521 ..."


In [41]:
# invalid geometry check
valid_col = input_gdf.is_valid.apply(lambda x: "VALID" if x else "REPAIRED")
try:
    assert sum(input_gdf.is_valid) == len(input_gdf)
except:
    error_count += 1
    warnings.warn('Not all input geometries are valid')
    error_msg += "Not all input geometries are valid" + "\n"
    print("Attempt to repair geometry: ")
    repaired_geom = input_gdf.make_valid()
    # not fixable geometries
    # in pipeline these will be reduced to Point
    print(f"Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries")
    error_msg += f"Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries" + "\n"
    repaired_geom.loc[repaired_geom.geom_type == 'GeometryCollection'] == input_gdf.geometry.loc[repaired_geom.geom_type == 'GeometryCollection']
    valid_col.loc[repaired_geom.geom_type == 'GeometryCollection'] == "INVALID"
    
input_gdf['ValidGeometry'] = valid_col

C:\Users\sky\AppData\Local\Temp\ipykernel_200020\3967708979.py:7: UserWarning: Not all input geometries are valid
  warnings.warn('Not all input geometries are valid')


Attempt to repair geometry: 
Found 0 unrepairable geometries


In [43]:
# Empty ACTIVITY_END check
try:
    assert sum(input_gdf.EndDate.isna()) == 0
except:
    error_count += 1
    
    warnings.warn("Not all data have valid activity end")
    error_msg += "Not all data have valid activity end" + "\n"
    
    print("    NA activity end data:")
    display(input_gdf[input_gdf.EndDate.isna()])
    
    # export to excel 

    NA activity end data:


C:\Users\sky\AppData\Local\Temp\ipykernel_200020\3327092817.py:7: UserWarning: Not all data have valid activity end
  warnings.warn("Not all data have valid activity end")


,Org_Public,ID,Name,Objective,Shape_Length,Shape_Area,Activity,Status,StartDate,EndDate,BroadVegType,Year,GISACRES,TreatedAcres,geometry,ValidGeometry
0,Private,19-FP-NEU-1037,Upper Yuba Fuelbreak Collaborative Project,Fuel Break,33975.765768,1.970798e+06,None,Proposed,NaT,NaT,Timber,NaN,486.994905,NaN,"MULTIPOLYGON (((-111056.954 158950.399, -11106...",VALID
1,Private,19-FH-NEU-012,Yuba Foothills Healthy Forest Project,Fuel Reduction,8207.653473,2.438554e+05,None,Proposed,NaT,NaT,Brush,NaN,60.257987,NaN,"MULTIPOLYGON (((-113243.967 152838.34, -113220...",VALID
2,Private,19-FH-NEU-012,Yuba Foothills Healthy Forest Project,Fuel Reduction,82074.564178,4.857025e+06,None,Proposed,NaT,NaT,Timber,NaN,1200.196965,NaN,"MULTIPOLYGON (((-100357.557 150779.573, -10034...",REPAIRED
3,Private,2-15EX-793-PLU,LaPorte Small Trees 1038j Pilot Exemption,Fuel Reduction,12777.804346,7.768582e+05,None,Exists,NaT,NaT,Timber,2015.0,191.965849,NaN,"MULTIPOLYGON (((-83607.363 187218.861, -83595....",VALID
4,Private,2-15EX-112-PLU,Lee Summit Mastication-Stevens Funds,Fuel Reduction,2812.367368,3.287648e+05,None,Exists,NaT,NaT,Timber,2016.0,81.239553,NaN,"MULTIPOLYGON (((-63511.137 207474.86, -63508.6...",VALID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4414,Private,SPI-3594,LYONS,Fuel Break,723.895953,1.372441e+04,Thinning (Mechanical),Planned,NaT,NaT,Timber,NaN,3.391371,NaN,"MULTIPOLYGON (((-13774.86 9748.775, -13752.907...",VALID
4417,Private,SPI-3597,West Lyons FB,Fuel Break,2050.454485,2.005421e+05,Thinning (Mechanical),Planned,NaT,NaT,Timber,NaN,49.554964,NaN,"MULTIPOLYGON (((-14574.249 10094.021, -14560.9...",VALID
4419,Private,SPI-3599,West Lyons FB,Fuel Break,1733.441534,4.452454e+04,Thinning (Mechanical),Planned,NaT,NaT,Timber,NaN,11.002237,NaN,"MULTIPOLYGON (((-13775.462 11658.16, -13775.63...",VALID
4421,Private,SPI-3601,Lyons Lake,Fuel Break,3816.725839,9.777358e+04,Thinning (Mechanical),Planned,NaT,NaT,Timber,NaN,24.160340,NaN,"MULTIPOLYGON (((-13773.343 10344.614, -13773.2...",VALID


In [45]:
# percent of data having activity status as COMPLETE
percent_complete = sum(input_gdf.Status == 'Complete')/len(input_gdf)
try:
    assert percent_complete > COMPLETE_THRESH
except:
    error_count += 1
    
    warnings.warn("Data does not pass activity status complete threshold")
    error_msg += "Data does not pass activity status complete threshold" + "\n"
    
    print(f"    {percent_complete}% of activity is COMPLETE")
    error_msg += f"    {percent_complete}% of activity is COMPLETE" + "\n"

In [46]:
# 0 activity quantity check
try:
    assert sum(input_gdf.GISACRES <= 0) == 0
except:
    error_count += 1
    
    warnings.warn("Exist ACTIVITY_QUANTITY = 0 case")
    error_msg += "Exist ACTIVITY_QUANTITY = 0 case" + "\n"
    print("    0 activity quantity data:")
    display(input_gdf[input_gdf.GISACRES <= 0])

In [47]:
del input_gdf
gc.collect()

2417

## CalTrans

In [57]:
caltrans_input_gdb_path = config_inputs['sources']['caltrans']['input']['gdb_path']
road_activity_layer_name = config_inputs['sources']['caltrans']['input']['road_activity_layer_name']
road_treatment_layer_name = config_inputs['sources']['caltrans']['input']['road_treatment_layer_name']

In [58]:
CALTRANS_ROAD_ACTIVITY_COLUMNS = [
    'Resp_District', 'Work_Order_Number', 'Begin_County', 'End_County', 'Route_Num',
    'Route', 'Route_Suffix', 'From_PM_Prefix', 'From_PM', 'From_PM_Suffix', 'From_PM_C',
    'To_PM_Prefix', 'To_PM', 'To_PM_Suffix', 'To_PM_C', 'Production_Quantity', 'UOM',
    'Maintenance_Type', 'Comments', 'Activity', 'Activity_Description', 'Highway_ID',
    'IMMS_Unit_ID', 'Fiscal_Year', 'From_Miles', 'To_Miles', 'WorkBeginDate',
    'WorkEndDate', 'ACTIVITY_NAME', 'PRIMARY_FUNDING_SOURCE_NAME',
    'PRIMARY_FUNDING_ORG_NAME', 'SECONDARY_FUNDING_SOURCE_NAME',
    'SECONDARY_FUNDING_ORG__NAME', 'TERTIARY_FUNDING_SOURCE_NAME',
    'TERTIARY_FUNDING_ORG_NAME', 'ADMINISTERING_ORG_NAME', 'IMPLEMENTING_ORG_NAME',
    'ACTIVITY_STATUS', 'BROAD_VEGETATION_TYPE', 'RESIDUE_QUANTITY', 'RESIDUE_FATE',
    'ACTIVITY_PERCENT_COMPLETE', 'ActivityID', 'Calendar_Year',
    'District', 'Route_C'
]

CALTRANS_ROAD_TREATMENT_COLUMNS = [
    'District_txt', 'District_num', 'County', 'Route_num', 'Route_txt',
    'Route_suffix', 'Highway_ID', 'FREQUENCY', 'SUM_Production_Quantity',
    'Ownership_Group', 'Primary_Objective', 'Secondary_Objective',
    'Tertiary_Objective', 'Estimated_Retreatment_Date', 'Treatment_Status',
    'Calendar_Year', 'Shape_Length', 'geometry'
]

In [59]:
# check file type
valid_file_types = ['gdb']
try: 
    assert caltrans_input_gdb_path[-3:] in valid_file_types
except: 
    error_count += 1
    warnings.warn("CALTRANS input is not the correct file type") 
    error_msg += "CALTRANS input is not the correct file type" + "\n"

    print("    Required file type: ", valid_file_types)
    print("    Input file type: ", caltrans_input_gdb_path[-3:])
    error_msg += "    Required file type: " + str(valid_file_types) + "\n"
    error_msg += "    Input file type: " + str(caltrans_input_gdb_path[-3:]) + "\n"

In [60]:
# check input layers
input_layers_dict = gpd.list_layers(caltrans_input_gdb_path).set_index('name')
# check if all input layer names are correct, exist in input gdb file
try:
    assert road_activity_layer_name in set(input_layers_dict.index)
    assert road_treatment_layer_name in set(input_layers_dict.index)
except:
    error_count += 1
    warnings.warn("Not all input layers exist")
    error_msg += "Not all input layers exist" + "\n"
    
    print("    Existing layer names: ", list(input_layers_dict.index))
    error_msg += "    Provided layer names: " + str([road_activity_layer_name, road_treatment_layer_name]) + "\n"
    error_msg += "    Existing layer names: " + str(input_layers_dict.index) + "\n"

### CalTrans Activity

In [65]:
# load data to gdf
input_gdf = gpd.read_file(caltrans_input_gdb_path,
                         driver='OpenFileGDB',
                         layer=road_activity_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [66]:
# schema check
try:
    assert len(set(CALTRANS_ROAD_ACTIVITY_COLUMNS) - set(input_gdf.columns)) == 0
except:
    error_count += 1
    warnings.warn("Not all required fields are present in input")
    error_msg += "Not all required fields are present in input" + "\n"
    
    print("    Missing required fields: ", set(CALTRANS_ROAD_ACTIVITY_COLUMNS) - set(input_gdf.columns))
    error_msg += "    Missing required fields: "+ str(set(CALTRANS_ROAD_ACTIVITY_COLUMNS) - set(input_gdf.columns)) + "\n"

    Missing required fields:  {'ACTIVITY_NAME', 'District', 'ADMINISTERING_ORG_NAME', 'WorkEndDate', 'TERTIARY_FUNDING_SOURCE_NAME', 'WorkBeginDate', 'TERTIARY_FUNDING_ORG_NAME', 'SECONDARY_FUNDING_ORG__NAME', 'IMPLEMENTING_ORG_NAME', 'RESIDUE_FATE', 'SECONDARY_FUNDING_SOURCE_NAME', 'Route_C', 'Route', 'ACTIVITY_PERCENT_COMPLETE', 'PRIMARY_FUNDING_ORG_NAME', 'BROAD_VEGETATION_TYPE', 'Resp_District', 'RESIDUE_QUANTITY', 'ACTIVITY_STATUS', 'PRIMARY_FUNDING_SOURCE_NAME', 'Fiscal_Year'}


C:\Users\sky\AppData\Local\Temp\ipykernel_200020\4086647401.py:6: UserWarning: Not all required fields are present in input
  warnings.warn("Not all required fields are present in input")


In [78]:
input_gdf.columns

Index(['Resp__District', 'Work_Order_Number', 'Activity',
       'Activity_Description', 'IMMS_Unit_ID', 'Highway_ID', 'Begin_County',
       'End_County', 'Route_Txt', 'Route_Suffix', 'Route_Num',
       'From_PM_Prefix', 'From_PM', 'From_PM_Suffix', 'To_PM_Prefix', 'To_PM',
       'To_PM_Suffix', 'From_Miles', 'To_Miles', 'Maintenance_Type',
       'Production_Quantity', 'UOM', 'Comments', 'Activity_Name',
       'Primary_Funding_Source_Name', 'Primary_Funding_Org_Name',
       'Secondary_Funding_Source_Name', 'Secondary_Funding_Source_Org',
       'Tertiary_Funding_Source_Name', 'Tertiary_Funding_Org_Name',
       'Administering_Org_Name', 'Implementing_Org_Name', 'Activity_Status',
       'Broad_Vegetation_Type', 'Residue_Quantity', 'Residue_Fate',
       'Activity_Percent_Complete', 'ActivityID', 'Task_Force_Activity_Desc',
       'Calendar_Year', 'District_Num', 'From_PM_C', 'To_PM_C', 'Fiscal_Yr',
       'Work_Begin_Date', 'Work_End_Date'],
      dtype='object')

0            7094704-7094704-DN-101-M0-C20040
1            7157588-7157588-DN-101-M0-C20040
2            7188054-7188054-DN-101-M0-C20040
3            7222583-7222583-DN-101-M0-C20040
4            7234737-7234737-DN-101-M0-C20040
                         ...                 
27838       7437268-7437268-LA-101-1.3-C21040
27839    7345549-7345549-CC-004-R19.08-C22040
27840      7446028-7446028-SF-101-1.24-C22040
27841    7226982-7226982-ALA-238-R14.5-C20040
27842      7480337-7480337-SD-005-R1.0-C22040
Length: 27843, dtype: object

In [83]:
# uid check
try:
    #assert len(input_gdf.GlobalID.unique()) == len(input_gdf)
    uid = input_gdf[['Work_Order_Number', 'Activity_Name']].apply(lambda row: str(row['Work_Order_Number'])+'-'+
                                            str(row['Activity_Name']), axis=1)
    assert len(uid.unique()) == len(input_gdf)
    
except:
    error_count += 1

    warnings.warn("Exist duplicate unique ID")
    error_msg += "Exist duplicate unique ID" + "\n"
    
    display(uid[uid.duplicated()])
    error_msg += str(uid[uid.duplicated()]) + "\n"

C:\Users\sky\AppData\Local\Temp\ipykernel_200020\1198424051.py:11: UserWarning: Exist duplicate unique ID
  warnings.warn("Exist duplicate unique ID")


3295     7167689-7167689-ALA-580-R28.5-E11040
8894        7278064-7278064-RIV-010-31-C32140
18231      7270315-7270315-SCL-130-3.1-E11040
dtype: object

In [84]:
# Empty ACTIVITY_END check
try:
    assert sum(input_gdf.Work_End_Date.isna()) == 0
except:
    error_count += 1
    
    warnings.warn("Not all data have valid activity end")
    error_msg += "Not all data have valid activity end" + "\n"
    
    print("    NA activity end data:")
    display(input_gdf[input_gdf.Work_End_Date.isna()])
    
    # export to excel 

In [87]:
# percent of data having activity status as COMPLETE
percent_complete = sum(input_gdf.Activity_Status == 'Complete')/len(input_gdf)
try:
    assert percent_complete > COMPLETE_THRESH
except:
    error_count += 1
    
    warnings.warn("Data does not pass activity status complete threshold")
    error_msg += "Data does not pass activity status complete threshold" + "\n"
    
    print(f"    {percent_complete}% of activity is COMPLETE")
    error_msg += f"    {percent_complete}% of activity is COMPLETE" + "\n"

In [88]:
# 0 activity quantity check
try:
    assert sum(input_gdf.Production_Quantity <= 0) == 0
except:
    error_count += 1
    
    warnings.warn("Exist ACTIVITY_QUANTITY = 0 case")
    error_msg += "Exist ACTIVITY_QUANTITY = 0 case" + "\n"
    print("    0 activity quantity data:")
    display(input_gdf[input_gdf.Production_Quantity <= 0])

In [97]:
del input_gdf
gc.collect()

4790

### CalTrans Treatment

In [90]:
# check if input layer geometry types are correct
try:
    assert input_layers_dict.loc[road_treatment_layer_name, 'geometry_type'] in ['LineString', 'MultiLineString']
except:
    error_count += 1
    warnings.warn("Input geometry type incorrect")
    error_msg += "Input geometry type incorrect" + "\n"
    
    print("    Valid geom types: ", ['Polygon', 'MultiPolygon'])
    print("    Input geom types: ", input_layers_dict.loc[ifprs_input_layer_name, 'geometry_type'])
    error_msg += "    Valid geom types: " + str(['LineString', 'MultiLineString']) + "\n"
    error_msg += "    Input geom types: " + input_layers_dict.loc[ifprs_input_layer_name, 'geometry_type'] + "\n"    

In [91]:
# load data to gdf
input_gdf = gpd.read_file(caltrans_input_gdb_path,
                         driver='OpenFileGDB',
                         layer=road_treatment_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured MultiLineString' is converted to 'MultiLineString'
  return ogr_read(


In [92]:
# schema check
try:
    assert len(set(CALTRANS_ROAD_TREATMENT_COLUMNS) - set(input_gdf.columns)) == 0
except:
    error_count += 1
    warnings.warn("Not all required fields are present in input")
    error_msg += "Not all required fields are present in input" + "\n"
    
    print("    Missing required fields: ", set(CALTRANS_ROAD_TREATMENT_COLUMNS) - set(input_gdf.columns))
    error_msg += "    Missing required fields: "+ str(set(CALTRANS_ROAD_TREATMENT_COLUMNS) - set(input_gdf.columns)) + "\n"

    Missing required fields:  {'Route_num', 'District_num'}


C:\Users\sky\AppData\Local\Temp\ipykernel_200020\408619185.py:6: UserWarning: Not all required fields are present in input
  warnings.warn("Not all required fields are present in input")


In [95]:
# uid check
try:
    #assert len(input_gdf.GlobalID.unique()) == len(input_gdf)
    uid = input_gdf[['Highway_ID', 'Calendar_Year']].apply(lambda row: str(row['Highway_ID'])+'-'+
                                        str(row['Calendar_Year']), axis=1)
    assert len(uid.unique()) == len(input_gdf)
    
except:
    error_count += 1

    warnings.warn("Exist duplicate unique ID")
    error_msg += "Exist duplicate unique ID" + "\n"
    
    display(uid[uid.duplicated()])
    error_msg += str(list(uid[uid.duplicated()])) + "\n"

In [96]:
# invalid geometry check
valid_col = input_gdf.is_valid.apply(lambda x: "VALID" if x else "REPAIRED")
try:
    assert sum(input_gdf.is_valid) == len(input_gdf)
except:
    error_count += 1
    warnings.warn('Not all input geometries are valid')
    error_msg += "Not all input geometries are valid" + "\n"
    print("Attempt to repair geometry: ")
    repaired_geom = input_gdf.make_valid()
    # not fixable geometries
    # in pipeline these will be reduced to Point
    print(f"Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries")
    error_msg += f"Found {sum(repaired_geom.geom_type == 'GeometryCollection')} unrepairable geometries" + "\n"
    repaired_geom.loc[repaired_geom.geom_type == 'GeometryCollection'] == input_gdf.geometry.loc[repaired_geom.geom_type == 'GeometryCollection']
    valid_col.loc[repaired_geom.geom_type == 'GeometryCollection'] == "INVALID"
    
input_gdf['ValidGeometry'] = valid_col

In [ ]:
del input_gdf
gc.collect()

In [99]:
error_msg

"Input geometry type incorrect\nNot all required fields are present in input\n    Missing required fields: {'Organization', 'OBJECTID'}\nExist duplicate unique ID\n['19-FH-NEU-012', '2-15EX-112-PLU', '2-15EX-112-PLU', '2-15-018-PLU', '2-18-062-PLU', '2-18-062-PLU', '2-18-053-TEH', '2-17-037-TEH', '2-14-085-PLU', '2-14-085-PLU', '2-15-013-PLU', '2-16-051-PLU', '2-16-052-PLU', 'MR1_100', 'MR1_50', '2-16-041-TEH.', '2-16-052-PLU', '2-14-086 PLU', '2-17-062-PLU', '88', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '34', '35', '36', '37', '38', '39', '40', '41', '1-15EX-273MEN', '1-15EX-273MEN', '1-15EX-273MEN', '1-15EX-273MEN', '1-16EX-280MEN', '1-16EX-280MEN', '1-16EX-280MEN', '1-16EX-280MEN', '1-19EX-00324-ME', 'VegMgmt 178', 'VegMgmt 178', 'VegMgmt 178', 'VegMgmt 178', 'VegMgmt 178', 'VegMgmt 178', '1-21EX-00578-ME', '1-21EX-00578-ME', '1-21EX-00578-ME', '1-21EX-00578-ME', '1-20-00170-MEN', '749', '749', '749', '749', '969', '969', '969', '899', 